In [5]:
import os
import numpy as np
from matplotlib import animation
import matplotlib.pyplot as plt
from IPython import display

In [6]:
BASE_DIR = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/train'

def normalize_range(data, bounds):
    """Maps data to the range [0, 1]."""
    return (data - bounds[0]) / (bounds[1] - bounds[0])

def get_false_color(band11,band14,band15):
    _T11_BOUNDS = (243, 303)
    _CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
    _TDIFF_BOUNDS = (-4, 2)
    
    r = normalize_range(band15 - band14, _TDIFF_BOUNDS)
    g = normalize_range(band14 - band11, _CLOUD_TOP_TDIFF_BOUNDS)
    b = normalize_range(band14, _T11_BOUNDS)
    false_color = np.clip(np.stack([r, g, b], axis=2), 0, 1)
    return false_color


def get_bands(no, base_dir):
    record_id = os.listdir(base_dir)[no]

    with open(os.path.join(base_dir, record_id, 'band_11.npy'), 'rb') as f:
        band11 = np.load(f)
    with open(os.path.join(base_dir, record_id, 'band_14.npy'), 'rb') as f:
        band14 = np.load(f)
    with open(os.path.join(base_dir, record_id, 'band_15.npy'), 'rb') as f:
        band15 = np.load(f)
    with open(os.path.join(base_dir, record_id, 'human_pixel_masks.npy'), 'rb') as f:
        mask= np.load(f)
    return  band11, band14, band15, mask




In [7]:
no=0

In [72]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
no += 1

band11, band14, band15, mask = get_bands(no, BASE_DIR)
false_color = get_false_color(band11, band14, band15)

im = ax[0].imshow(false_color[..., 0])
def draw(i):
    im.set_array(false_color[..., i])
    return [im]
ax[1].imshow(mask[..., 0])

anim = animation.FuncAnimation(fig, draw, frames=false_color.shape[-1], interval=200, blit=True)
plt.close()
display.HTML(anim.to_jshtml())

